# Dataframe Generation for Model Input
* Author: Callie Clark
* Date updated: 6/29/2021

In [1]:
import pandas as pd
import requests
import datetime as dt
import numpy as np

import dataframe_generation_files as gen

## Format Incident Data

In [2]:
df=gen.format_file('Chicago_Data/Crimes_-_2014.zip','Crimes_-_2014.csv',timezone="America/Chicago")
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Timestamp
0,9470797,HX123824,2014-01-22,031XX S ASHLAND AVE,0460,BATTERY,SIMPLE,RESTAURANT,True,False,...,59,08B,1166178.0,1883833.0,2014,05/24/2017 03:50:24 PM,41.836816,-87.665724,"(41.836816162, -87.665724279)",2014-01-22 16:02:00-06:00
5,9729405,HX379028,2014-08-08,044XX W JACKSON BLVD,4651,OTHER OFFENSE,SEX OFFENDER: FAIL REG NEW ADD,RESIDENCE,True,False,...,26,26,1146739.0,1898315.0,2014,05/21/2017 03:53:28 PM,41.876949,-87.736685,"(41.876949072, -87.736685059)",2014-08-08 10:50:00-05:00
6,9510600,HX165568,2014-02-26,037XX N MONTICELLO AVE,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,16,05,1151395.0,1924539.0,2014,05/19/2017 03:49:49 PM,41.948820,-87.718900,"(41.948820056, -87.718899701)",2014-02-26 07:40:00-06:00
12,9539289,HX192580,2014-03-20,111XX S STATE ST,0460,BATTERY,SIMPLE,GAS STATION,True,False,...,49,08B,1178229.0,1831287.0,2014,05/15/2017 03:51:52 PM,41.692359,-87.623097,"(41.692358646, -87.623096746)",2014-03-20 19:25:00-05:00
15,9801642,HX450761,2014-09-13,070XX S MERRILL AVE,1725,OFFENSE INVOLVING CHILDREN,CONTRIBUTE CRIM DELINQUENCY JUVENILE,OTHER,True,True,...,43,20,1191762.0,1858706.0,2014,05/09/2017 03:51:05 PM,41.767282,-87.572664,"(41.767282129, -87.572663824)",2014-09-13 14:00:00-05:00


In [3]:
df['Location Description'].unique()

array(['RESTAURANT', 'RESIDENCE', 'GAS STATION', 'OTHER', 'STREET',
       'APARTMENT', 'PARKING LOT/GARAGE(NON.RESID.)',
       'MEDICAL/DENTAL OFFICE', 'RESIDENTIAL YARD (FRONT/BACK)',
       'RESIDENCE PORCH/HALLWAY', 'BANK', 'VEHICLE NON-COMMERCIAL',
       'SCHOOL, PRIVATE, GROUNDS', 'SMALL RETAIL STORE',
       'CTA TRACKS - RIGHT OF WAY', 'SCHOOL, PUBLIC, BUILDING', 'ALLEY',
       'GROCERY FOOD STORE', 'AIRPORT VENDING ESTABLISHMENT', 'SIDEWALK',
       'RESIDENCE-GARAGE', 'CONVENIENCE STORE', 'VEHICLE-COMMERCIAL',
       'CAR WASH', 'FACTORY/MANUFACTURING BUILDING',
       'TAVERN/LIQUOR STORE', 'BAR OR TAVERN', 'CTA BUS', 'PARK PROPERTY',
       'HOTEL/MOTEL', 'NURSING HOME/RETIREMENT HOME', 'DEPARTMENT STORE',
       'CURRENCY EXCHANGE', 'COMMERCIAL / BUSINESS OFFICE',
       'CTA BUS STOP', 'APPLIANCE STORE', 'CTA TRAIN', 'WAREHOUSE',
       'TAXICAB', 'CTA STATION', 'CONSTRUCTION SITE',
       'AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA',
       'HOSPITAL BUILDING/GROU

## Add Murder Data 
https://home.chicagopolice.org/wp-content/uploads/2014/12/2011-Murder-Report.pdf
(removed from CFS)
TO DO: Clean up and create a function

In [4]:
import geopandas as gpd
from random_timestamp import random_timestamp
import random
from shapely.geometry import Point
from numpy import asarray


def generate_random(number, polygon):
    points = []
    timestamps=[]
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(asarray(pnt))
            timestamps.append(random_timestamp(year=2014))
            #lat_long.append(asarray(pnt))
    return points, timestamps

fp = "Simulation_Data/Chicago_PB.shp"
polys  = gpd.read_file(fp)

In [5]:
murders_df=pd.read_csv('Chicago_Data/Chicago_Murders_District.csv')
murders_df['Average Murders']=murders_df[['2010 Murders','2011 Murders']].mean(axis=1)
murders_2014=murders_df[['District','Average Murders']]#.set_index('District')
murders_2014['District']=murders_2014['District'].astype(str)
polys=polys.merge(murders_2014,how='left',left_on='dist_num',right_on='District').drop(columns=['District']).fillna(0)
polys.head()

<ipython-input-5-3ac121af723d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  murders_2014['District']=murders_2014['District'].astype(str)


,dist_label,dist_num,geometry,Average Murders
0,17TH,17,"POLYGON ((-87.71067 41.99737, -87.71067 41.997...",7.0
1,20TH,20,"POLYGON ((-87.66029 41.99092, -87.66029 41.990...",3.5
2,31ST,31,"POLYGON ((-87.82818 41.98384, -87.82816 41.983...",0.0
3,31ST,31,"POLYGON ((-87.83365 41.97535, -87.83366 41.974...",0.0
4,19TH,19,"POLYGON ((-87.64492 41.96973, -87.64431 41.969...",2.0


In [6]:
murder_by_district={}
murders_loc=[]
timestamp_murders=[]
for index_loc in polys.index:
    district=polys.iloc[index_loc,1]
    murder_num=polys.iloc[index_loc,3]
    #print(district,murder_num)
    points, timestamps=generate_random(int(murder_num), polys.geometry[index_loc])
    #print(len(timestamps),len(points))
    #murder_by_district[int(district)]=points
    timestamp_murders=timestamp_murders+timestamps
    murders_loc=murders_loc+points

    

In [7]:
len(murders_loc)

417

In [8]:
df_2 = pd.DataFrame(0, index=np.arange(len(timestamp_murders)), columns=df.columns)
df_2['Timestamp']=timestamp_murders
df_2['Timestamp']=df_2['Timestamp'].dt.tz_localize("America/Chicago",nonexistent='shift_forward',ambiguous='NaT')
df_2['Location']=murders_loc
df_2['Latitude']=df_2.apply((lambda x: x['Location'][1]),axis=1)
df_2['Longitude']=df_2.apply((lambda x: x['Location'][0]),axis=1)
df_2['Date']=df_2.apply((lambda x: x['Timestamp'].date()),axis=1)
df_2['FBI Code']=["01A" for i in range(len(timestamp_murders))]
df_2.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Timestamp
0,0,0,2014-06-30,0,0,0,0,0,0,0,...,0,01A,0,0,0,0,41.973012,-87.739809,"[-87.73980930331898, 41.97301206357757]",2014-06-30 15:57:03-05:00
1,0,0,2014-10-06,0,0,0,0,0,0,0,...,0,01A,0,0,0,0,41.995960,-87.747493,"[-87.74749270633848, 41.995960277991365]",2014-10-06 23:11:50-05:00
2,0,0,2014-04-11,0,0,0,0,0,0,0,...,0,01A,0,0,0,0,41.972042,-87.727733,"[-87.72773276650925, 41.972041679714856]",2014-04-11 08:52:02-05:00
3,0,0,2014-04-18,0,0,0,0,0,0,0,...,0,01A,0,0,0,0,41.971671,-87.709761,"[-87.70976055049587, 41.971671295376716]",2014-04-18 16:23:15-05:00
4,0,0,2014-07-14,0,0,0,0,0,0,0,...,0,01A,0,0,0,0,41.957253,-87.744018,"[-87.7440179166415, 41.957252615060675]",2014-07-14 00:51:11-05:00


In [9]:
df=pd.concat([df,df_2]) #add murders to original DataFrame

In [10]:
gen.check_lat_long(df)

Data has Latitude and Longitude


In [11]:
violent_crime=["01A",'02','03','04A','04B']
property_crime=['05','06','07','09']
index_crime=["01A",'02','03','04A','04B','05','06','07','09']
nonindex_crime=['01B', '08A', '08B','10','11','12','13','14','15','16','17','18','19','20','22','24','26']
nonviolent_crime=nonindex_crime+property_crime
    
df=gen.classify_response_type(df, crime_responder='Police', crime_types=index_crime, UCRcol='FBI Code' )
#TODO may want to indicate the crime type in csv name
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Timestamp,Police
0,9470797,HX123824,2014-01-22,031XX S ASHLAND AVE,0460,BATTERY,SIMPLE,RESTAURANT,1,0,...,08B,1166178.0,1883833.0,2014,05/24/2017 03:50:24 PM,41.836816,-87.665724,"(41.836816162, -87.665724279)",2014-01-22 16:02:00-06:00,0
5,9729405,HX379028,2014-08-08,044XX W JACKSON BLVD,4651,OTHER OFFENSE,SEX OFFENDER: FAIL REG NEW ADD,RESIDENCE,1,0,...,26,1146739.0,1898315.0,2014,05/21/2017 03:53:28 PM,41.876949,-87.736685,"(41.876949072, -87.736685059)",2014-08-08 10:50:00-05:00,0
6,9510600,HX165568,2014-02-26,037XX N MONTICELLO AVE,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,0,0,...,05,1151395.0,1924539.0,2014,05/19/2017 03:49:49 PM,41.948820,-87.718900,"(41.948820056, -87.718899701)",2014-02-26 07:40:00-06:00,1
12,9539289,HX192580,2014-03-20,111XX S STATE ST,0460,BATTERY,SIMPLE,GAS STATION,1,0,...,08B,1178229.0,1831287.0,2014,05/15/2017 03:51:52 PM,41.692359,-87.623097,"(41.692358646, -87.623096746)",2014-03-20 19:25:00-05:00,0
15,9801642,HX450761,2014-09-13,070XX S MERRILL AVE,1725,OFFENSE INVOLVING CHILDREN,CONTRIBUTE CRIM DELINQUENCY JUVENILE,OTHER,1,1,...,20,1191762.0,1858706.0,2014,05/09/2017 03:51:05 PM,41.767282,-87.572664,"(41.767282129, -87.572663824)",2014-09-13 14:00:00-05:00,0


In [12]:
df=gen.set_priority(df)
df=gen.set_number_units(df)

In [20]:
print('Percentage "Violent" crime')
len(df[df["Priority"]==1])/len(df)

0.0835322903536739

In [21]:
print('Percentage "Index" crime')
len(df[df["Police"]==1])/len(df)

0.3990398161192259

In [12]:
df=gen.log_norm_service_time(df, average_time=23.2,s=0.18)

Stats of dist: min 10.263997411914367 max 51.86408454298283 mean 23.575461084649987


In [13]:
#Double Service Times for Murders
df.loc[df['FBI Code']=='01A','Service Time']=df.loc[df['FBI Code']=='01A','Service Time']*2

In [14]:
df.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Year,Updated On,Latitude,Longitude,Location,Timestamp,Police,Priority,#Units,Service Time
412,0,0,2014-04-03,0,0,0,0,0,0,0,...,0,0,41.893426,-87.755826,"[-87.75582563609291, 41.89342560275382]",2014-04-03 17:37:11-05:00,1,1,3,0 days 00:38:00
413,0,0,2014-03-25,0,0,0,0,0,0,0,...,0,0,41.898624,-87.762520,"[-87.76252009224683, 41.89862435567867]",2014-03-25 00:33:56-05:00,1,1,2,0 days 00:46:00
414,0,0,2014-04-10,0,0,0,0,0,0,0,...,0,0,41.869584,-87.756632,"[-87.75663229856792, 41.86958379621768]",2014-04-10 10:25:53-05:00,1,1,3,0 days 00:44:00
415,0,0,2014-08-28,0,0,0,0,0,0,0,...,0,0,41.867269,-87.760674,"[-87.76067397564788, 41.86726861997915]",2014-08-28 07:07:39-05:00,1,1,3,0 days 00:40:00
416,0,0,2014-03-30,0,0,0,0,0,0,0,...,0,0,41.879479,-87.771441,"[-87.77144115113096, 41.87947858765941]",2014-03-30 16:05:25-05:00,1,1,3,0 days 00:40:00


In [15]:
len(df[df['Priority']==1])/len(df)#violent

0.0835322903536739

In [16]:
(len(df[df['Priority']==1])+len(df[df['Priority']==2]))/len(df)#index

0.3990398161192259

## Create Seasonal Profiles
*  Crime frequency and type varies by season

In [17]:
season_map={}
df=gen.create_seasons(df,season_map)
df=gen.calculate_frequency(df)
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Latitude,Longitude,Location,Timestamp,Police,Priority,#Units,Service Time,season,Frequency
0,9470797,HX123824,2014-01-22,031XX S ASHLAND AVE,0460,BATTERY,SIMPLE,RESTAURANT,1,0,...,41.836816,-87.665724,"(41.836816162, -87.665724279)",2014-01-22 16:02:00-06:00,0,3,1,0 days 00:30:00,winter,622
5,9729405,HX379028,2014-08-08,044XX W JACKSON BLVD,4651,OTHER OFFENSE,SEX OFFENDER: FAIL REG NEW ADD,RESIDENCE,1,0,...,41.876949,-87.736685,"(41.876949072, -87.736685059)",2014-08-08 10:50:00-05:00,0,3,2,0 days 00:24:00,summer,807
6,9510600,HX165568,2014-02-26,037XX N MONTICELLO AVE,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,0,0,...,41.948820,-87.718900,"(41.948820056, -87.718899701)",2014-02-26 07:40:00-06:00,1,2,2,0 days 00:17:00,winter,659
12,9539289,HX192580,2014-03-20,111XX S STATE ST,0460,BATTERY,SIMPLE,GAS STATION,1,0,...,41.692359,-87.623097,"(41.692358646, -87.623096746)",2014-03-20 19:25:00-05:00,0,3,1,0 days 00:30:00,spring,771
15,9801642,HX450761,2014-09-13,070XX S MERRILL AVE,1725,OFFENSE INVOLVING CHILDREN,CONTRIBUTE CRIM DELINQUENCY JUVENILE,OTHER,1,1,...,41.767282,-87.572664,"(41.767282129, -87.572663824)",2014-09-13 14:00:00-05:00,0,3,1,0 days 00:23:00,summer,667


In [18]:
df.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Latitude,Longitude,Location,Timestamp,Police,Priority,#Units,Service Time,season,Frequency
412,0,0,2014-04-03,0,0,0,0,0,0,0,...,41.893426,-87.755826,"[-87.75582563609291, 41.89342560275382]",2014-04-03 17:37:11-05:00,1,1,3,0 days 00:38:00,spring,710
413,0,0,2014-03-25,0,0,0,0,0,0,0,...,41.898624,-87.762520,"[-87.76252009224683, 41.89862435567867]",2014-03-25 00:33:56-05:00,1,1,2,0 days 00:46:00,spring,701
414,0,0,2014-04-10,0,0,0,0,0,0,0,...,41.869584,-87.756632,"[-87.75663229856792, 41.86958379621768]",2014-04-10 10:25:53-05:00,1,1,3,0 days 00:44:00,spring,793
415,0,0,2014-08-28,0,0,0,0,0,0,0,...,41.867269,-87.760674,"[-87.76067397564788, 41.86726861997915]",2014-08-28 07:07:39-05:00,1,1,3,0 days 00:40:00,summer,722
416,0,0,2014-03-30,0,0,0,0,0,0,0,...,41.879479,-87.771441,"[-87.77144115113096, 41.87947858765941]",2014-03-30 16:05:25-05:00,1,1,3,0 days 00:40:00,spring,734


In [19]:
seasonal_profiles={}
seasonal_profiles=gen.generate_seasonal_profiles(df,seasonal_profiles,quantile_list=[0.5,0.95],iterations=2, number_profiles=1)
seasonal_profiles
#To Do fix allowing seasonal and all worst case to repeat

{'summer_0.5': [datetime.date(2014, 8, 3), datetime.date(2014, 8, 16)],
 'summer_0.95': [datetime.date(2014, 7, 11), datetime.date(2014, 7, 11)],
 'fall_0.5': [datetime.date(2014, 12, 3), datetime.date(2014, 11, 8)],
 'fall_0.95': [datetime.date(2014, 9, 27), datetime.date(2014, 9, 26)],
 'spring_0.5': [datetime.date(2014, 4, 11), datetime.date(2014, 4, 18)],
 'spring_0.95': [datetime.date(2014, 6, 17), datetime.date(2014, 6, 17)],
 'winter_0.5': [datetime.date(2014, 2, 9), datetime.date(2014, 1, 19)],
 'winter_0.95': [datetime.date(2014, 3, 18), datetime.date(2014, 3, 14)],
 'summer_worst_case': [datetime.date(2014, 8, 1)],
 'fall_worst_case': [datetime.date(2014, 10, 1)],
 'spring_worst_case': [datetime.date(2014, 6, 1)],
 'winter_worst_case': [datetime.date(2014, 1, 1)]}

In [20]:
#To Do fix allowing seasonal and all worst case to repeat -->currently hardcoding
#seasonal_profiles['summer_worst_case']=[dt.date(2014, 9, 1)]

In [21]:
seasonal_profiles

{'summer_0.5': [datetime.date(2014, 8, 3), datetime.date(2014, 8, 16)],
 'summer_0.95': [datetime.date(2014, 7, 11), datetime.date(2014, 7, 11)],
 'fall_0.5': [datetime.date(2014, 12, 3), datetime.date(2014, 11, 8)],
 'fall_0.95': [datetime.date(2014, 9, 27), datetime.date(2014, 9, 26)],
 'spring_0.5': [datetime.date(2014, 4, 11), datetime.date(2014, 4, 18)],
 'spring_0.95': [datetime.date(2014, 6, 17), datetime.date(2014, 6, 17)],
 'winter_0.5': [datetime.date(2014, 2, 9), datetime.date(2014, 1, 19)],
 'winter_0.95': [datetime.date(2014, 3, 18), datetime.date(2014, 3, 14)],
 'summer_worst_case': [datetime.date(2014, 8, 1)],
 'fall_worst_case': [datetime.date(2014, 10, 1)],
 'spring_worst_case': [datetime.date(2014, 6, 1)],
 'winter_worst_case': [datetime.date(2014, 1, 1)]}

In [22]:
daily_profiles={}
gen.quantile_daily_profiles(df, daily_profiles,quantiles=[0.25,0.50,0.75,0.95],iterations=2)
#gen.worst_case_profiles(df, daily_profiles,number_profiles=2)
daily_profiles

{'all_data_0.25': [datetime.date(2014, 2, 28), datetime.date(2014, 12, 27)],
 'all_data_0.5': [datetime.date(2014, 5, 20), datetime.date(2014, 6, 8)],
 'all_data_0.75': [datetime.date(2014, 5, 21), datetime.date(2014, 8, 20)],
 'all_data_0.95': [datetime.date(2014, 7, 27), datetime.date(2014, 6, 29)]}

## Generate Incident Profiles and send to csvs

In [23]:
gen.create_csv(daily_profiles,df,folder='Chicago_Data/Daily_Profiles')

In [24]:
gen.create_csv(seasonal_profiles,df,folder='Chicago_Data/Daily_Profiles')